In [2]:
import pandas as pd

In [4]:
pbyears = range(1980,2019)

ColNames=['Pos', 'Player', 'Conf', 'Team', 'Age', 'Yrs', 'G', 'GS', 'PassingCmp', 'PassingAtt', 'PassingYds', 'PassingTD', 
 'PassingInt', 'RushingAtt', 'RushingYds', 'RushingTD', 'ReceivingRec',
 'ReceivingYds', 'ReceivingTD', 'Tackles', 'Int', 'Sacks', 'All-Pro Teams'] 

pb_df = pd.DataFrame(columns = ColNames)


for x in pbyears:
    url = f'https://www.pro-football-reference.com/years/{x}/probowl.htm'
    pb_list = pd.read_html(url) # returns list
    tempDF = pd.DataFrame(pb_list[0])
#     tempDF.insert(0, 'Pro-Bowl Year', x)
    tempDF.columns = ColNames
    if x == min(pbyears):
        pb_df = tempDF
    else:
        pb_df = pb_df.append(tempDF, ignore_index=True)
del tempDF
   
#Drop the College Stats column because it is just the text 'College Stats' - use bs to get the actual href and merge in subsequent steps
# draftDF.drop(['CollegeStats'], axis=1, inplace = True)   

In [5]:
pb_df['Number of Pro-Bowls'] = 1

In [8]:
pb_app_df = pb_df.drop(columns = ['Conf', 'Team', 'Age', 'Yrs', 'G', 'GS', 'PassingCmp', 'PassingAtt',
    'PassingYds', 'PassingTD', 'PassingInt', 'RushingAtt', 'RushingYds', 'RushingTD', 'ReceivingRec', 'ReceivingYds',
    'ReceivingTD', 'Tackles', 'Int', 'Sacks', 'All-Pro Teams'])
pb_app_df['Player'] = pb_app_df['Player'].str.replace(r'%$', '')
pb_pivot = pb_app_df.groupby(['Player'])['Number of Pro-Bowls'].sum() 
pb_pivot_df = pd.DataFrame(pb_pivot).reset_index()
pb_pivot_df['Pro-Bowler'] = 'Y'
pb_pivot_df.Player = pb_pivot_df.Player.str.strip('+')
pb_pivot_df

,Player,Number of Pro-Bowls,Pro-Bowler
0,A.J. Bouye,1,Y
1,A.J. Duhe,1,Y
2,A.J. Green,7,Y
3,Aaron Donald,5,Y
4,Aaron Glenn,3,Y
...,...,...,...
1570,Zach Miller,1,Y
1571,Zach Thomas,7,Y
1572,Zack Martin,5,Y
1573,Zak DeOssie,2,Y


In [9]:
# commit to sqlite database
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
engine = create_engine('sqlite:///../data/fbdata.sqlite')
pb_pivot_df.to_sql('probowl', con=engine, if_exists='replace', index=False)